In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
with h5py.File('../data/processed_data.h5', 'r') as hf:
    X_train = hf['X_train'][:]
X_train = np.array(X_train, dtype=np.float32)

In [5]:
print(X_train.shape)

(24959, 56, 10)
